In [1]:
import napari
from macrohet import dataio, tile, visualise
import os, glob
from tqdm.auto import tqdm
import pandas as pd
import shutil

In [2]:
os.getpid()

721706

In [4]:
def copy_files(file_basenames, source_dir, destination_dir, progress_file='progress.txt'):
    # Create destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)
    print('Creating progress file')
    
    # Path for the progress file
    progress_file_path = os.path.join(destination_dir, progress_file)
    
    # Read the progress file if it exists to know where to resume
    if os.path.exists(progress_file_path):
        with open(progress_file_path, 'r') as f:
            completed_files = set(f.read().splitlines())
    else:
        completed_files = set()

    print('Checking destination directory for already copied files')
    # Count files in destination directory to approximate progress
    N_files = len(os.listdir(destination_dir))
    print(f'{N_files} files already copied')

    # Assume the first N_files in the destination directory match the first N_files in file_basenames
    # and add them to the completed_files set, if they are not already present.
    for basename in file_basenames[:N_files]:
        if basename not in completed_files:
            completed_files.add(basename)
            with open(progress_file_path, 'a') as f:
                f.write(basename + '\n')

    print('Starting copy job')
    # Initialize tqdm for visual progress feedback
    with tqdm(total=len(file_basenames)-N_files, desc='Copying files') as pbar:
        for basename in file_basenames:
            # Construct source and destination file paths
            src_path = os.path.join(source_dir, basename)
            dst_path = os.path.join(destination_dir, basename)
            
            # Ensure the destination directory exists
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            
            # Skip if already copied
            if basename in completed_files:
                pbar.update(1)
                continue

            # Copy the file
            shutil.copy2(src_path, dst_path)
            
            # Mark this file as completed by adding it to the progress file
            with open(progress_file_path, 'a') as f:
                f.write(basename + '\n')
            
            # Update the progress bar
            pbar.update(1)


In [3]:
expt_IDs = [ 'ND0001', 'ND0000','ND0002', 'ND0003']
missing_fns = []
for expt_ID in tqdm(expt_IDs): 
    
    metadata_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/Index*xml')[0]
    
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    
    file_basenames = metadata['URL']
    
    source_directory = f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/'
    dest_directory = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/Images/'
    for file_basename in tqdm(file_basenames, desc = f'iterating over files in position {expt_ID}'):
        
        source_path = os.path.join(source_directory, file_basename)
        dest_path = os.path.join(dest_directory, file_basename)
        
        if not os.path.exists(source_path):
            print(f'{file_basename} not found in {source_directory}')
            missing_fns.append(source_path)
            
            # print(full_path, ' transferring')
            # shutil.copy2(source_path, source_path)


  0%|          | 0/4 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


iterating over files in position ND0001:   0%|          | 0/1020600 [00:00<?, ?it/s]

...

iterating over files in position ND0002:   0%|          | 0/388620 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


iterating over files in position ND0003:   0%|          | 0/349272 [00:00<?, ?it/s]

In [4]:
missing_fns

[]

In [ ]:
dest_directory = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/Images/'

In [39]:
expt_IDs = [ 'ND0001',]# 'ND0000','ND0002', 'ND0003']
missing_fns = []
for expt_ID in tqdm(expt_IDs): 
    
    metadata_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/Index*xml')[0]
    
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    
    file_basenames = metadata['URL']
    
    source_directory = f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/'
    dest_directory = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/Images/'
    for file_basename in tqdm(file_basenames, desc = f'iterating over files in position {expt_ID}'):
        
        source_path = os.path.join(source_directory, file_basename)
        dest_path = os.path.join(dest_directory, file_basename)

        if not os.path.exists(dest_directory):
            # print(full_path, ' transferring')
            # shutil.copy2(source_path, source_path)


  0%|          | 0/1 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


iterating over files in position ND0001:   0%|          | 0/1020600 [00:00<?, ?it/s]

In [43]:
source_path

'/mnt/SYNO/macrohet_syno/ND0001/acquisition/Images/r06c12f09p03-ch3sk300fk1fl1.tiff'

In [42]:
dest_path

'/mnt/NEMO/home/users/dayn/macrohet_nemo/ND0001/acquisition/Images/r06c12f09p03-ch3sk300fk1fl1.tiff'

In [45]:
# Initialize a list to hold filenames that caused exceptions
error_files = []

for file_basename in tqdm(file_basenames, desc=f'iterating over files in position {expt_ID}'):
    source_path = os.path.join(source_directory, file_basename)
    dest_path = os.path.join(dest_directory, file_basename)

    if not os.path.exists(dest_path):
        try:
            # Attempt to copy the file from source to destination
            shutil.copy2(source_path, dest_path)  # Corrected to copy to dest_path instead of source_path to source_path
        except SameFileError:
            # If a SameFileError is encountered, add the file to the error_files list
            error_files.append(file_basename)
        except Exception as e:
            # Optionally handle other exceptions
            print(f"An error occurred with file {file_basename}: {e}")
            error_files.append(file_basename)

iterating over files in position ND0001:   0%|          | 0/1020600 [00:00<?, ?it/s]

In [33]:
os.path.exists(full_path)

True

In [38]:
missing_fns

[]

In [ ]:
'/mnt/NEMO/home/users/dayn/macrohet_nemo/'

In [35]:
for file_basename in tqdm(file_basenames, desc = f'iterating over files in position {expt_ID}'):
        
        full_path = os.path.join(source_directory, file_basename)
        
        if not os.path.exists(full_path):
            print(full_path, ' MISSING')
            missing_fns.append(full_path)

iterating over files in position ND0001:   0%|          | 0/1020600 [00:00<?, ?it/s]

In [29]:
expt_IDs = [ 'ND0000', 'ND0001','ND0002', 'ND0003']
missing_fns = []
for expt_ID in tqdm(expt_IDs): 
    
    metadata_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/Index*xml')[0]
    
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    
    file_basenames = metadata['URL']
    
    source_directory = f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/'

    for file_basename in tqdm(file_basenames, desc = f'iterating over files in position {expt_ID}'):
        
        full_path = os.path.join(source_directory, file_basename)
        
        if not os.path.exists(full_path):
            print(full_path, ' MISSING')
            missing_fns.append(full_path)
    break

  0%|          | 0/4 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


iterating over files in position ND0000:   0%|          | 0/243648 [00:00<?, ?it/s]

In [30]:
missing_fns

[]

In [15]:
%%time
srcs = glob.glob('/mnt/NEMO/home/users/dayn/macrohet_nemo/macrohet_images/ND0000/Images/*.tiff')


KeyboardInterrupt



In [17]:
len(srcs)

243648

In [24]:
import shutil

In [26]:
for src in tqdm(srcs):
    
    dst = src.replace('macrohet_images/ND0000', 'ND0000/acquisition')
    if os.path.exists(dst):
        continue
    else: 
        shutil.move(src, dst)

  0%|          | 0/243648 [00:00<?, ?it/s]

In [22]:
dst

'/mnt/NEMO/home/users/dayn/macrohet_nemo/ND0000/acquisition/Images/r02c04f03p03-ch1sk348fk1fl1.tiff'

In [4]:
expt_IDs = ['ND0001', 'ND0000', 'ND0002', 'ND0003']

for expt_ID in tqdm(expt_IDs): 
    
    metadata_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/Index*xml')[0]
    
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    
    file_basenames = metadata['URL']
    
    source_directory = f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/Images/'
    
    destination_directory = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/Images/'
    
    copy_files(file_basenames, source_directory, destination_directory)


  0%|          | 0/4 [00:00<?, ?it/s]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Creating progress file
Checking destination directory for already copied files
294957 files already copied
Starting copy job


Copying files:   0%|          | 0/725643 [00:00<?, ?it/s]

BlockingIOError: [Errno 11] Resource temporarily unavailable: '/mnt/NEMO/home/users/dayn/macrohet_nemo/ND0001/acquisition/Images/r04c07f02p03-ch3sk221fk1fl1.tiff'

In [ ]:
missing_fns = []
for fn in tqdm(metadata['URL']):
    if not os.path.exists(os.path.join(base_dir, f'Images/{fn}')):
        missing_fns.append(fn)

In [ ]:
len(missing_fns)

In [ ]:
85041/158607

In [ ]:
len(metadata)- len(missing_fns)

In [ ]:
metadata_fn

In [ ]:
base_dir

In [ ]:
missing_fns = []
for fn in tqdm(metadata['URL']):
    if not os.path.exists(os.path.join(base_dir, f'acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images/{fn}')):
        missing_fns.append(fn)

In [ ]:
len(missing_fns)

In [ ]:
import re

In [ ]:
# Regular expression pattern to extract rows and columns
pattern = r'r(\d+)c(\d+)f(\d+)p(\d+)-ch(\d+)sk(\d+)fk(\d+)fl(\d+)'

# Extract rows and columns from filenames and store them in a set to ensure uniqueness
unique_values = set()
data = []
for filename in missing_fns:
    match = re.search(pattern, filename)
    if match:
        row = match.group(1)
        column = match.group(2)
        plane = match.group(4)
        time = match.group(6)
        channel = match.group(5)
        data.append((row, column, plane, time, channel))

# Create a set of unique rows and columns
unique_rows = set()
unique_columns = set()

for row, column, plane, time, channel in data:
    unique_rows.add(row)
    unique_columns.add(column)

# Check for missing planes, times, or channels in rows and columns
missing_planes = set()
missing_times = set()
missing_channels = set()

for row, column, plane, time, channel in data:
    if plane not in missing_planes:
        missing_planes.add(plane)
    if time not in missing_times:
        missing_times.add(time)
    if channel not in missing_channels:
        missing_channels.add(channel)

# Print the unique rows and columns without missing planes, times, or channels
print("Unique Rows:")
for row in unique_rows:
    if all(plane in missing_planes for plane in unique_rows):
        print(row)

print("Unique Columns:")
for column in unique_columns:
    if all(channel in missing_channels for channel in unique_columns):
        print(column)

In [ ]:
len(unique_values)

In [ ]:
unique_values

In [ ]:
len(metadata)- len(missing_fns)

In [ ]:
len(missing_fns)/len(metadata)*100

In [ ]:
75*2*3*9

In [ ]:
r03c01f01p01-ch1sk1fk1fl1

In [ ]:
%%timeit
fns = glob.glob('/mnt/DATA/macrohet/PS0000/acquisition/Images/*r03c04*ch*')

In [ ]:
len(fns)

In [ ]:
%%timeit
fns = glob.glob('/mnt/NEMO/home/users/dayn/macrohet_nemo/PS0000/acquisition/Images/*.tif*')

In [ ]:
len(fns)

In [ ]:
%%timeit
fns = glob.glob('/mnt/SYNO/macrohet_syno/PS0000/acquisition/Images/*.tif*')

In [ ]:
print(len(fns))

In [ ]:
449/10.1 # syno times quicker than nemo ???

In [ ]:
164/9.67 # syno times quicker than my local harddrive!?

In [ ]:
488/164 # local times quicker than nemo!?

In [ ]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

In [ ]:
acq_ID = row, column = (3, 4)

In [ ]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

In [ ]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
images

In [ ]:
%%time 
### this is NEMO
images = images.compute().compute()

In [ ]:
%%time 
### this is SYNO
images = images.compute().compute()

In [ ]:
images.shape

In [ ]:
viewer = napari.Viewer(title = f'testing speed of image load fgrom syno {expt_ID}')

viewer.add_image(images, channel_axis = 1)

In [ ]:
import btrack
scale_factor = 1/5.04

In [ ]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/testing_{row, column}.h5'), 
                                   'r', 
                                   obj_type='obj_type_1'
                                   ) as reader:
#             writer.write_objects(objects)
        tracks = reader.tracks
        segmentation = reader.segmentation

In [ ]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks ,scale = (1/scale_factor,1/scale_factor))

In [ ]:
filtered_tracks = [t for t in tracks if len(t)>15]

In [ ]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)

In [ ]:
viewer.add_tracks(napari_tracks, scale = (20, 1/scale_factor,1/scale_factor,))

In [ ]:
viewer.add_labels(recolored_segmentation)

In [ ]:
.shape

In [ ]:
viewer.add_image(images[14], channel_axis=0)

In [ ]:
print()

In [ ]:
base_dir

In [ ]:
for fn in glob.glob(os.path.join(base_dir, 'labels/testing*')):
    
    with btrack.io.HDF5FileHandler(fn, 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
    #             writer.write_objects(objects)
            tracks = reader.tracks
            # segmentation = reader.segmentation
    filtered_tracks = [t for t in tracks if len(t)>15]
    napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
    viewer.add_tracks(napari_tracks, scale = (1/scale_factor,1/scale_factor,), name = os.path.basename(fn))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Optional: Define a list of colors if you want specific colors
colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
color_idx = 0

for fn in glob.glob(os.path.join(base_dir, 'labels/testing*')):
    with btrack.io.HDF5FileHandler(fn, 'r', obj_type='obj_type_1') as reader:
        tracks = reader.tracks

    # Plotting with varying colors and transparency
    plt.hist([len(track) for track in tracks], color=colors[color_idx % len(colors)], alpha=0.1)

    # Increment color index for next iteration
    color_idx += 1

# Set y-axis limit
plt.ylim(0, 1000)

# Display the plot
plt.show()

In [ ]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# model = models.Cellpose(gpu=True, model_type='cyto')
model_path = '/mnt/DATA/macrohet/upstream_development/segmentation/cellpose_training/models/models/macrohet_seg'
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)
def segment(img, diameter = 30, channels = [0,0]):
    masks, flows, styles, diams = model.eval(img, diameter=diameter, channels=channels,
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

In [ ]:
img = images[0]

In [ ]:
img.shape

In [ ]:
masks = segment(img, diameter = None)

In [ ]:
model = models.Cellpose(gpu=True, model_type='cyto')
output = model.eval(img, diameter = 300)#, diameter=diameter, channels=channels,
                                             # flow_threshold=None, cellprob_threshold=0)

In [ ]:
output[0].shape
masks = output[0]

In [ ]:
viewer.add_labels(masks)

In [ ]:
print()

In [ ]:
img

In [ ]:
import torch
print(torch.__version__)

In [ ]:
from segment_anything import SamPredictor, sam_model_registry


In [ ]:
import numpy as np

In [ ]:
np.save('t0_ND2.npy', cut_images[0])